# Setup 

In [2]:
# Weights and Biases
!pip install -q wandb
# Tensorflow
!pip install -q tensorflow


[notice] A new release of pip available: 22.3.1 -> 23.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.3.1 -> 23.0
[notice] To update, run: pip install --upgrade pip


In [3]:
from keras.models import Sequential, Model
from keras.layers import Input, LSTM, Concatenate, Dense, BatchNormalization, LeakyReLU
from keras.activations import tanh
from tensorflow.keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers.schedules import ExponentialDecay
import pandas as pd
import numpy as np

2023-02-01 21:00:20.847008: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [11]:
import wandb
from wandb.keras import WandbCallback
wandb.login(key='3cae81eb56be3190be5bb48c571e69933071df69')

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /Users/hjalmarjacobvinje/.netrc


True

# Load data

In [15]:
file = "../data/processed_data/2022.csv"
df = pd.read_csv(file)
display(df.head(30))
df.head(20)

val_set = 100000
num_features = 4
time_steps = 10

train_y = df['Quote_date'][:-val_set].to_numpy()
val_y = df['Quote_date'][-val_set:].to_numpy()

array = df.to_numpy()

train_x = [array[:-val_set][:,-time_steps:].reshape(len(array[:-val_set]),time_steps,1), array[:-val_set][:,:num_features]]
val_x = [array[-val_set:][:,-time_steps:].reshape(len(array[-val_set:]),time_steps,1), array[-val_set:][:,:num_features]]

print(train_x[0].shape)
print(train_x[1].shape)
print(val_x[0].shape)
print(val_x[1].shape)

,Unnamed: 0,Quote_date,Expire_date,Price,Underlying_last,Strike,TTM,Underlying_1,Underlying_2,Underlying_3,Underlying_4,Underlying_5,Underlying_6,Underlying_7,Underlying_8,Underlying_9,Underlying_10,R
0,97145,2022-01-18,2022-01-19,3378.25,4578.88,1200.0,1,4662.85,4662.21,4660.64,4726.55,4713.53,4669.85,4676.41,4696.25,4700.64,4793.19,0.05
1,97146,2022-01-18,2022-01-19,3178.25,4578.88,1400.0,1,4662.85,4662.21,4660.64,4726.55,4713.53,4669.85,4676.41,4696.25,4700.64,4793.19,0.05
2,97147,2022-01-18,2022-01-19,2978.25,4578.88,1600.0,1,4662.85,4662.21,4660.64,4726.55,4713.53,4669.85,4676.41,4696.25,4700.64,4793.19,0.05
3,97148,2022-01-18,2022-01-19,2778.25,4578.88,1800.0,1,4662.85,4662.21,4660.64,4726.55,4713.53,4669.85,4676.41,4696.25,4700.64,4793.19,0.05
4,97149,2022-01-18,2022-01-19,2678.25,4578.88,1900.0,1,4662.85,4662.21,4660.64,4726.55,4713.53,4669.85,4676.41,4696.25,4700.64,4793.19,0.05
5,97150,2022-01-18,2022-01-19,2578.25,4578.88,2000.0,1,4662.85,4662.21,4660.64,4726.55,4713.53,4669.85,4676.41,4696.25,4700.64,4793.19,0.05
6,97151,2022-01-18,2022-01-19,2478.25,4578.88,2100.0,1,4662.85,4662.21,4660.64,4726.55,4713.53,4669.85,4676.41,4696.25,4700.64,4793.19,0.05
7,97152,2022-01-18,2022-01-19,2378.25,4578.88,2200.0,1,4662.85,4662.21,4660.64,4726.55,4713.53,4669.85,4676.41,4696.25,4700.64,4793.19,0.05
8,97153,2022-01-18,2022-01-19,2278.25,4578.88,2300.0,1,4662.85,4662.21,4660.64,4726.55,4713.53,4669.85,4676.41,4696.25,4700.64,4793.19,0.05
9,97154,2022-01-18,2022-01-19,2178.25,4578.88,2400.0,1,4662.85,4662.21,4660.64,4726.55,4713.53,4669.85,4676.41,4696.25,4700.64,4793.19,0.05


(1739894, 10, 1)
(1739894, 4)
(100000, 10, 1)
(100000, 4)


# Model construction

In [21]:
def create_model(config):
    '''Builds an LSTM-MLP model of minimum 2 layers sequentially from a given config dictionary'''
    underlying_history = Input((config.LSTM_timesteps,1))
    bs_vars = Input((config.Num_features,))

    model = Sequential()

    model.add(LSTM(
        units = config.LSTM_units,
        activation = tanh,
        input_shape = (config.LSTM_timesteps, 1),
        return_sequences = True
    ))

    for _ in range(config.LSTM_layers - 2):
        model.add(LSTM(
            units = config.LSTM_units,
            activation = tanh,
            return_sequences = True
        ))
    
    model.add(LSTM(
        units = config.LSTM_units,
        activation = tanh,
        return_sequences = False
    ))

    layers = Concatenate()([model(underlying_history), bs_vars])
    
    for _ in range(config.MLP_layers - 1):
        layers = Dense(config.MLP_units)(layers)
        layers = BatchNormalization(momentum=config.Bn_momentum)(layers)
        layers = LeakyReLU()(layers)

    #create the output layer
    output = Dense(1, activation='relu')(layers)

    #create exponentially decaying learning rate schedule
    lr_schedule = ExponentialDecay(
        initial_learning_rate = config.Lr,
        decay_steps = int(len(train_x)/config.Minibatch_size),
        decay_rate=config.Lr_decay
    )

    #compile the model
    model = Model(inputs=[underlying_history, bs_vars], outputs=output)
    model.compile(loss='mse', optimizer=Adam(learning_rate=lr_schedule))

    return model

# Hyperparameter search setup

In [22]:
# Configuring the sweep hyperparameter search space
sweep_configuration = {
    'method': 'random',
    'name': 'LSTM-MLP v.1.0 test data',
    'metric': {
        'goal': 'minimize', 
        'name': 'val_loss'
		},
    'parameters': {
        'LSTM_units': {
            'values': [32, 64, 96, 128]},
        'MLP_units': {
            'values': [32, 64, 96, 128]},
        'LSTM_timesteps': {
            'values': [10]},
        'LSTM_layers': {
            'distribution': 'int_uniform',
            'max': 6, 'min': 4},
        'MLP_layers': {
            'distribution': 'int_uniform',
            'max': 6, 'min': 4},
        'Bn_momentum': {
            'distribution': 'uniform',
            'max': 0.4, 'min': 0},
        'Lr': {
            'distribution': 'uniform',
            'max': 0.005, 'min': 0.0005},
        'Lr_decay': {
            'distribution': 'uniform',
            'max': 1, 'min': 0.9},        
        'Minibatch_size': {
            'values': [1024, 2048, 4096]},
        'Min_delta': {
            'value': 1},
        'Patience': {
            'value': 20},
        'Num_features': {
            'value': 4},
    }
}

# Initialize sweep and creating sweepID

# If new sweep, uncomment the line below and comment the line after it
#sweep_id = wandb.sweep(sweep=sweep_configuration, project='Deep learning for option pricing - test area') 
sweep_id = '98bxt6oq'

# Run hyperparameter search

In [23]:
def trainer(train_x = train_x, train_y = train_y, val_x = val_x, val_y = val_y, config = None, project = None):
    # Initialize a new wandb run
    with wandb.init(config=config, project = project):

        # If called by wandb.agent, as below,
        # this config will be set by Sweep Controller
        config = wandb.config

        model = create_model(config)

        early_stopping = EarlyStopping(
            monitor='val_loss',
            mode='min',
            min_delta = config.Min_delta,
            patience = config.Patience,
        )
        
        wandb_callback = WandbCallback(
            monitor='val_loss',
            mode='min',
            save_model=False
        )
        
        model.fit(
            train_x,
            train_y,
            batch_size = config.Minibatch_size,
            validation_data = (val_x, val_y),
            epochs = 1000,
            callbacks = [early_stopping, wandb_callback] 
        )

## Run full sweep

In [24]:
wandb.agent(sweep_id=sweep_id, function=trainer, project='Deep learning for option pricing - test area', count = 10)

wandb: Agent Starting Run: qi5863el with config:
wandb: 	Bn_momentum: 0.0042450897520058374
wandb: 	LSTM_layers: 4
wandb: 	LSTM_timesteps: 20
wandb: 	LSTM_units: 96
wandb: 	Lr: 0.003814355437640067
wandb: 	Lr_decay: 0.9453983300912104
wandb: 	MLP_layers: 5
wandb: 	MLP_units: 64
wandb: 	Min_delta: 1
wandb: 	Minibatch_size: 1024
wandb: 	Num_features: 4
wandb: 	Patience: 20
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Run qi5863el errored: ValueError('Failed to convert a NumPy array to a Tensor (Unsupported object type float).')
wandb: ERROR Run qi5863el errored: ValueError('Failed to convert a NumPy array to a Tensor (Unsupported object type float).')
wandb: Agent Starting Run: xu7u3yy7 with config:
wandb: 	Bn_momentum: 0.28450434007205505
wandb: 	LSTM_layers: 4
wandb: 	LSTM_timesteps: 20
wandb: 	LSTM_units: 128
wandb: 	Lr: 0.002378711371909496
wandb: 	Lr_decay: 0.916297516076408
wandb: 	MLP_layers: 6
wandb: 	MLP_units: 128
wandb: 	Min_delta: 1
wandb: 	Minibatch_size: 2048
wandb: 	Num_features: 4
wandb: 	Patience: 20
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


wandb: Ctrl + C detected. Stopping sweep.


## Single run

In [25]:
config = {
    'LSTM_units': 4,
    'MLP_units': 100,
    'LSTM_timesteps': 20,
    'LSTM_layers': 4,
    'MLP_layers': 4,
    'Bn_momentum': 0.9,
    'Lr': 1e-4,
    'Lr_decay': 1,
    'Minibatch_size': 10,
    'Min_delta': 0.1,
    'Patience': 20,
    'Num_features': 4,
    'Architecture': 'LSTM-MLP v.0.1',
    'Dataset': 'Test data',
}
trainer(config = config, project = 'Deep learning for option pricing - test area')

wandb: WARNING Ignored wandb.init() arg project when running a sweep.


Exception in thread Thread-43 (_run_job):
Traceback (most recent call last):
  File "/var/folders/wk/x86_p6511l95p594k6qnb98h0000gn/T/ipykernel_16718/85754476.py", line 24, in trainer
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/wandb/integration/keras/keras.py", line 174, in new_v2
    return old_v2(*args, **kwargs)
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/wandb/integration/keras/keras.py", line 174, in new_v2
    return old_v2(*args, **kwargs)
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/wandb/integration/keras/keras.py", line 174, in new_v2
    return old_v2(*args, **kwargs)
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/keras/utils/traceback_utils.py", line 70, in error_handler
    raise e.with_traceback(filtered_tb) from None
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packag

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type float).